# 📘 Chapter 12: Custom Models and Training with TensorFlow

Chapter ini membahas cara membuat model machine learning yang benar-benar fleksibel menggunakan API low-level TensorFlow. Kita tidak hanya menggunakan Keras seperti biasanya, tetapi membuat **loss function**, **metrics**, **layers**, bahkan **training loop** secara manual.

---

## 🎯 Tujuan Utama

- Memahami bagaimana cara kerja model machine learning dari bawah
- Membuat custom loss function, custom metric, dan custom layer
- Menulis loop pelatihan manual menggunakan `GradientTape`
- Menggunakan `tf.data` untuk mengelola pipeline data yang efisien
- Menyesuaikan training dengan callback buatan sendiri

---

## 🔧 Custom Loss Function

### ➕ Contoh: Huber Loss

Huber Loss merupakan gabungan dari MSE dan MAE, cocok digunakan ketika data memiliki outlier.

```python
def huber_fn(y_true, y_pred):
    ...


In [2]:
# CHAPTER 12: Custom Models and Training with TensorFlow (Gabungan Lengkap)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)

# -------------------------------------
# Custom Loss Function: Huber Loss
# -------------------------------------
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1.0
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# -------------------------------------
# Custom Metric Class
# -------------------------------------
class HuberMetric(keras.metrics.Metric):
    def __init__(self, name="huber_metric", threshold=1.0, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - 0.5
        losses = tf.where(is_small_error, squared_loss, linear_loss)
        self.total.assign_add(tf.reduce_sum(losses))
        self.count.assign_add(tf.cast(tf.size(error), tf.float32))

    def result(self):
        return self.total / self.count

    def reset_states(self):
        self.total.assign(0.0)
        self.count.assign(0.0)

# -------------------------------------
# Dummy Dataset
# -------------------------------------
X = tf.random.normal((1000, 1))
y = 3 * X + 2 + tf.random.normal((1000, 1))

# -------------------------------------
# Custom Training Loop with GradientTape
# -------------------------------------
model = keras.Sequential([layers.Dense(1, input_shape=[1])])
optimizer = keras.optimizers.SGD(learning_rate=0.01)

for epoch in range(10):
    for step in range(len(X) // 32):
        X_batch = X[step * 32: (step + 1) * 32]
        y_batch = y[step * 32: (step + 1) * 32]
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            loss = tf.reduce_mean(huber_fn(y_batch, y_pred))
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    print(f"[ManualLoop] Epoch {epoch + 1}, Loss: {loss.numpy():.4f}")

# -------------------------------------
# Training with tf.data Pipeline
# -------------------------------------
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(1)

model2 = keras.Sequential([layers.Dense(1, input_shape=[1])])
optimizer2 = keras.optimizers.SGD(learning_rate=0.01)

for epoch in range(10):
    for X_batch, y_batch in dataset:
        with tf.GradientTape() as tape:
            y_pred = model2(X_batch, training=True)
            loss = tf.reduce_mean(huber_fn(y_batch, y_pred))
        grads = tape.gradient(loss, model2.trainable_variables)
        optimizer2.apply_gradients(zip(grads, model2.trainable_variables))
    print(f"[tf.data] Epoch {epoch + 1}, Loss: {loss.numpy():.4f}")

# -------------------------------------
# Custom Layer
# -------------------------------------
class MyDenseLayer(keras.layers.Layer):
    def __init__(self, units=32, activation=None):
        super().__init__()
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)

# -------------------------------------
# Custom Model Using Custom Layer
# -------------------------------------
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden = MyDenseLayer(30, activation="relu")
        self.output_layer = MyDenseLayer(1)

    def call(self, inputs):
        x = self.hidden(inputs)
        return self.output_layer(x)

model_custom = MyModel()
model_custom.compile(loss=huber_fn, optimizer="sgd")
model_custom.fit(X, y, epochs=5)

# -------------------------------------
# Custom Callback
# -------------------------------------
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        ratio = logs["val_loss"] / logs["loss"]
        print(f"Epoch {epoch + 1}: val/train loss ratio = {ratio:.2f}")

# -------------------------------------
# Model with Custom Callback
# -------------------------------------
model_cb = keras.Sequential([layers.Dense(1, input_shape=[1])])
model_cb.compile(loss=huber_fn, optimizer="sgd")
model_cb.fit(X, y, epochs=5, validation_split=0.2, callbacks=[PrintValTrainRatioCallback()])


TensorFlow version: 2.18.0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[ManualLoop] Epoch 1, Loss: 1.9545
[ManualLoop] Epoch 2, Loss: 1.7886
[ManualLoop] Epoch 3, Loss: 1.6266
[ManualLoop] Epoch 4, Loss: 1.4690
[ManualLoop] Epoch 5, Loss: 1.3167
[ManualLoop] Epoch 6, Loss: 1.1704
[ManualLoop] Epoch 7, Loss: 1.0309
[ManualLoop] Epoch 8, Loss: 0.9002
[ManualLoop] Epoch 9, Loss: 0.7796
[ManualLoop] Epoch 10, Loss: 0.6704
[tf.data] Epoch 1, Loss: 1.9246
[tf.data] Epoch 2, Loss: 2.5335
[tf.data] Epoch 3, Loss: 3.1604
[tf.data] Epoch 4, Loss: 2.3626
[tf.data] Epoch 5, Loss: 1.8503
[tf.data] Epoch 6, Loss: 1.0719
[tf.data] Epoch 7, Loss: 0.3387
[tf.data] Epoch 8, Loss: 1.0078
[tf.data] Epoch 9, Loss: 1.4268
[tf.data] Epoch 10, Loss: 0.7995
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.5626
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4664 
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4118
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3888 
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2407 
Epoch